In [17]:
# Enter your script here.

import numpy as np
import pandas as pd
import csv
import re
import calendar
import datetime
format_str = '%Y-%m-%d' # The format
import functools 
import operator
import simplejson as json

def check_columns(data):
    for columns in data:
      print(columns + ":  " +  str(len(pd.unique(data[columns]))) )
      print(pd.unique(data[columns]))
      print('++++++++++++++++++')

def splitQuery(column):
    listOfthings = "'" + column.astype(str).str.cat(sep="', '") + "'"
    return listOfthings

In [23]:
dados_brutos = pd.read_csv('dados_brutos.csv')
dados_cidades = pd.read_csv('municipios.csv')
dados_estados = pd.read_csv('estados.csv')

dados_brutos

dados_geo = dados_cidades.merge(dados_estados,left_on='codigo_uf', right_on='codigo_uf',suffixes=('_cidade', '_estado'))

In [24]:
dados_geo['cidade_estado'] = dados_geo['nome_cidade'] + ' ' + dados_geo['nome_estado']

In [25]:
dados_brutos.columns = dados_brutos.columns.str.replace(' ', '_')

regex = r"\_\(\w*|\W*\)"
regex_under = r"(^\_)"

dict_para_novos_nomes = {}

for items in dados_brutos.columns:
    ##caixa baixa
    item_caixa_baixa = items.lower()
    ##tirar parenteses bizarros e underscore comeco
    novo_item = re.sub(regex,'',item_caixa_baixa)
    sem_under = re.sub(regex_under,'',novo_item)
    ##tirar co2 com superscript
    dict_para_novos_nomes[items] = sem_under.replace('co²','co2')

#renomear colunas aqui
dados_brutos.rename(columns = dict_para_novos_nomes,inplace=True)

In [26]:
pd.unique(dados_brutos['estado'])

array(['Pará', 'Minas Gerais', 'São Paulo ', 'Distrito Federal',
       'São Paulo', 'Paraná', 'Santa Catarina', 'Ceará', 'Goiás',
       'Amazonas', 'Rio de Janeiro', 'Rio Grande do Sul', 'Pernambuco',
       'Bahia', 'Espírito Santo'], dtype=object)

In [45]:
dados_brutos.replace(to_replace=r'\%', value='', regex=True, inplace=True)
dados_brutos.replace(to_replace = 'dockless', value = np.NaN, inplace=True)

lista_colunas_trocar = ['estações','mulheres','homens','15_até_29_anos','30_até_59_anos','acima_de_60']

dicionario_final = {}
dicionario_final['local'] = 'Brasil'
dicionario_final['sistema_total'] = []
dicionario_final['sistema_por_estado'] = []

dict_siglas = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}

def get_key(val,siglas): 
    for key, value in siglas.items(): 
         if val == value: 
             return key 
        
    return "key doesn't exist"

### dados agregados de sistemas

dados_brutos['estado'] = dados_brutos['estado'].str.rstrip()

for estados in pd.unique(dados_brutos['estado']):
    print(estados)
    dict_estado = {}
    dict_estado['estado'] = estados
    frame_estado = dados_brutos[dados_brutos['estado'] == estados]
    dict_estado['sistemas'] = []
    for sis_por_estado in pd.unique(frame_estado['sistema']):
        sigla = get_key(estados.rstrip(),dict_siglas)
        dicionario_final['sistema_total'].append(sis_por_estado + ' – ' + sigla)
        dict_estado['sistemas'].append(sis_por_estado + ' – ' + sigla)
    dicionario_final['sistema_por_estado'].append(dict_estado)

dicionario_final['operadora'] = pd.unique(dados_brutos['operadora']).tolist()

### dados agregados de estações, bicicletas, patinetes, viagens, distância
for col_errada in lista_colunas_trocar:
    dados_brutos[col_errada] = pd.to_numeric(dados_brutos[col_errada], errors='coerce')

lista_para_somar = ['estações','bicicletas_elétricas','bicicletas','patinetes_elétricos','viagens_diárias','média_distância_percorrida_por_dia','usuários','emissões_de_co2_evitadas']        

for col_para_somar in lista_para_somar:
    if np.isnan(dados_brutos[col_para_somar].sum(min_count=1)) == True:
        dicionario_final[col_para_somar] = 'Dados não disponíveis'
    else:
        dicionario_final[col_para_somar] = dados_brutos[col_para_somar].sum(min_count=1)

## dados agregados de dados demográficos
lista_demograficos = ['mulheres','homens','15_até_29_anos','30_até_59_anos','acima_de_60']

for col_porcentagem in lista_demograficos:
    lista_valores_totais = []
    print(col_porcentagem)
    for municipio in pd.unique(dados_brutos['cidade']):
        frame_municipio = dados_brutos[dados_brutos['cidade'] ==  municipio]
        if np.isnan(frame_municipio['usuários'].sum(min_count=1)) == True:
            dicionario_final[col_porcentagem] = 'Dados não disponíveis'
        else:
            for sistema in pd.unique(frame_municipio['sistema']):
                frame_sistema = frame_municipio[frame_municipio['sistema'] == sistema]
                if np.isnan(frame_sistema['usuários'].sum(min_count=1)) == True:
                    dicionario_final[col_porcentagem] = 'Dados não disponíveis'
                else:
                    for veiculo in pd.unique(frame_sistema['tipologia_do_veículo']):
                        frame_final = frame_sistema[frame_sistema['tipologia_do_veículo'] == veiculo]
#                         if np.isnan(frame_final['usuários']) == True:
                        if frame_final['usuários'].empty == True:
                            dicionario_final[col_porcentagem] = 'Dados não disponíveis'
                        else:
                            valor_absoluto = (frame_final['usuários'].values*frame_final[col_porcentagem].values)/100
                            print('======== valor abs ========')
                            print(valor_absoluto[0])
                            print(valor_absoluto)
                            lista_valores_totais.append(valor_absoluto[0])                        
    if not lista_valores_totais:
        dicionario_final[col_porcentagem] = 'Dados não disponíveis'
    else:
        print(lista_valores_totais)
        dicionario_final[col_porcentagem] = functools.reduce((lambda x, y: x * y), lista_valores_totais)

Pará
Minas Gerais
São Paulo
Distrito Federal
Paraná
Santa Catarina
Ceará
Goiás
Amazonas
Rio de Janeiro
Rio Grande do Sul
Pernambuco
Bahia
Espírito Santo
mulheres
======== valor abs ========
478.0293
[478.0293]
======== valor abs ========
14827.326
[14827.326]
======== valor abs ========
2986.3439999999996
[2986.344]
======== valor abs ========
28356.3504
[28356.3504]
======== valor abs ========
2800.8762
[2800.8762]
======== valor abs ========
28.0002
[28.0002]
======== valor abs ========
11426.5566
[11426.5566]
======== valor abs ========
1556.1984
[1556.1984]
======== valor abs ========
2034.1791999999998
[2034.1792]
======== valor abs ========
7979.963199999999
[7979.9632]
[478.0293, 14827.326, 2986.3439999999996, 28356.3504, 2800.8762, 28.0002, 11426.5566, 1556.1984, 2034.1791999999998, 7979.963199999999]
homens
======== valor abs ========
570.9707
[570.9707]
======== valor abs ========
21837.674
[21837.674]
======== valor abs ========
5449.656
[5449.656]
======== valor abs =======

In [46]:
def cidadePorc(frame,dici):
    if frame[col_por].isna().values.any():
        dici[col_por] = 'Dados não disponíveis'
    else:
        filter_frame = frame[~pd.isnull(frame['usuários'])]
        null_frame = frame[pd.isnull(frame['usuários'])]
        
        if len(filter_frame.index) == 1:
            dici[col_por] = filter_frame[col_por].values[0]
        elif len(filter_frame.index) > 1:
            total_usuario = filter_frame['usuários'].sum()
            total_porc = filter_frame[col_por].sum()
            valor_absoluto = (total_porc/total_usuario)*100
            dici[col_por] = valor_absoluto
        else:
            if len(null_frame) == 1:
                dici[col_por] = null_frame[col_por].values[0]
            else:
                frame_sistema = frame[frame['sistema'] == sistema]

                if frame_sistema[col_por].isna().values.any():
                    dici[col_por] = 'Dados não disponíveis'
                else:
                    filter_frame = frame_sistema[~pd.isnull(frame['usuários'])]
                    null_frame = frame_sistema[pd.isnull(frame_sistema['usuários'])]

                    if len(filter_frame.index) == 1:
                        dici[col_por] = filter_frame[col_por].values[0]
                    elif len(filter_frame.index) > 1:
                        total_usuario = filter_frame['usuários'].sum()
                        total_porc = filter_frame[col_por].sum()
                        valor_absoluto = (total_porc/total_usuario)*100
                        dici[col_por] = valor_absoluto
                    else:
                        if len(null_frame.index) == 1:
                            check_columns(null_frame)
                            dici[col_por] = null_frame[col_por].values[0]
                        else:
                            total_usuario = null_frame['usuários'].sum()
                            total_porc = null_frame[col_por].sum()
                            valor_absoluto = (total_porc/total_usuario)*100
                            dici[col_por] = valor_absoluto
            
    return dici[col_por]





def getPorcentagens(frame,dici):
    
    if frame[col_por].isna().values.any():
        dici[col_por] = 'Dados não disponíveis'
    else:
        filter_frame = frame[~pd.isnull(frame['usuários'])]
        null_frame = frame[pd.isnull(frame['usuários'])]
        if len(filter_frame.index) == 1:
            dici[col_por] = filter_frame[col_por].values[0]
        elif len(filter_frame.index) > 1:
            total_usuario = filter_frame['usuários'].sum()
            total_porc = filter_frame[col_por].sum()
            valor_absoluto = (total_porc/total_usuario)*100
            dici[col_por] = valor_absoluto
        else:
            if len(null_frame.index) == 1:
                check_columns(null_frame)
                dici[col_por] = null_frame[col_por].values[0]
            else:
                total_usuario = null_frame['usuários'].sum()
                total_porc = null_frame[col_por].sum()
                valor_absoluto = (total_porc/total_usuario)*100
                dici[col_por] = valor_absoluto
            
    return dici[col_por]

In [47]:
dicionario_final['municipios'] = []

lista_para_somar_2 = ['estações','bicicletas_elétricas','bicicletas','patinetes_elétricos','viagens_diárias','média_distância_percorrida_por_dia','usuários','emissões_de_co2_evitadas']
lista_demograficos_2 = ['mulheres','homens','15_até_29_anos','30_até_59_anos','acima_de_60']


## ler todas as cidades no arquivo, e para cada cidade, criar um dicionario/objeto
## aos poucos, adicionar propriedades únicas como nome e outras coisas mais

for cidade_item in pd.unique(dados_brutos['cidade']):
    ## criar corte nos dados só com a cidade selecionada no momento dentro do loop
    ## vamos fazer calculos em cima disso
    frame_cidade = dados_brutos[dados_brutos['cidade'] == cidade_item]
    print(cidade_item)
    dic_cidade = {}
    dic_cidade['cidade'] = cidade_item.rstrip()
    
    id_cidade_estado = cidade_item.rstrip() + ' ' + frame_cidade.iloc[0]['estado'].rstrip()
#     dic_cidade['estado'] = id_cidade_estado
    print(id_cidade_estado)
    frame_com_latlong = dados_geo[dados_geo['cidade_estado'] == id_cidade_estado]
    
    dic_cidade['lat'] = frame_com_latlong.iloc[0]['latitude']
    dic_cidade['long'] = frame_com_latlong.iloc[0]['longitude']
    
    dic_cidade['sistemas'] = []
    dic_cidade['modais'] = []
    
    ### dados gerais das cidades
    for col_para_somar in lista_para_somar_2:
        if np.isnan(frame_cidade[col_para_somar].sum(min_count=1)) == True:
            dic_cidade[col_para_somar] = 'Dados não disponíveis'
        else:
            dic_cidade[col_para_somar] = frame_cidade[col_para_somar].sum(min_count=1)
            
    ### dados demograficos das cidades
    for col_por in lista_demograficos_2:
        cidadePorc(frame_cidade,dic_cidade)
           
    ##########################################################################################
    ### dados agregados de sistemas
    for sistema in pd.unique(frame_cidade['sistema']):
        frame_sistema = frame_cidade[frame_cidade['sistema'] == sistema]
        dict_sistema = {}
        dict_sistema['sistema'] = sistema
        dict_sistema['cidade'] = dic_cidade['cidade']
#         dict_sistema['estado'] = dic_cidade['estado']        
        dict_sistema['lat'] = dic_cidade['lat']
        dict_sistema['long'] = dic_cidade['long']

        ### dados gerais dos sistemas
        for col_para_somar in lista_para_somar_2:
            if np.isnan(frame_sistema[col_para_somar].sum(min_count=1)) == True:
                dict_sistema[col_para_somar] = 'Dados não disponíveis'
            else:
                dict_sistema[col_para_somar] = frame_sistema[col_para_somar].sum(min_count=1)
        
        ### dados demograficos dos sistemas
        for col_por in lista_demograficos_2:
            getPorcentagens(frame_sistema,dict_sistema)
     
        ##########################################################################################
        ### Adicionar dicionario/objeto de cada veiculo dentro de cada sistema; passo final!
        dict_sistema['veiculos'] = []
        for veiculo in pd.unique(frame_sistema['tipologia_do_veículo']):
            print(veiculo)
            dict_veiculo = {}
            dict_veiculo['lat'] = dic_cidade['lat']
            dict_veiculo['long'] = dic_cidade['long']
            frame_veiculo = frame_sistema[frame_sistema['tipologia_do_veículo'] == veiculo]
            lista_colunas_veiculo = frame_veiculo.columns.values
            for colunas_linha in lista_colunas_veiculo:
                dict_veiculo[colunas_linha] = frame_veiculo.iloc[0][colunas_linha]
            dict_sistema['veiculos'].append(dict_veiculo)
        ##########################################################################################
        
        ### Adicionar sistemas e modais a sua cidade
        dic_cidade['sistemas'].append(dict_sistema)
    
    ##########################################################################################
    ### dados agregados de MODAIS
    for modal in pd.unique(frame_cidade['tipologia_do_veículo']):
        frame_modal = frame_cidade[frame_cidade['tipologia_do_veículo'] == modal]
        dict_modal = {}
        dict_modal['tipologia_do_veículo'] = modal
        dict_modal['cidade'] = dic_cidade['cidade']
#         dict_modal['estado'] = dic_cidade['estado']
        dict_modal['lat'] = dic_cidade['lat']
        dict_modal['long'] = dic_cidade['long']

        ### dados gerais dos modais
        for col_para_somar in lista_para_somar_2:
            if np.isnan(frame_modal[col_para_somar].sum(min_count=1)) == True:
                dict_modal[col_para_somar] = 'Dados não disponíveis'
            else:
                dict_modal[col_para_somar] = frame_modal[col_para_somar].sum(min_count=1)
        
        ### dados demograficos dos sistemas
        for col_por in lista_demograficos_2:
            getPorcentagens(frame_modal,dict_modal)
        
        dic_cidade['modais'].append(dict_modal)
        
    ##########################################################################################
    ### Adicionar cidades ao objeto final
    dic_cidade
    dicionario_final['municipios'].append(dic_cidade)

Belém
Belém Pará
Bicicleta
Belo Horizonte
Belo Horizonte Minas Gerais
Bicicleta
Bicicleta
Patinete Eletrico
Bertioga
Bertioga São Paulo
Bicicleta
Brasília
Brasília Distrito Federal
Bicicleta
Patinete Eletrico
Campinas
Campinas São Paulo
Bicicleta
Patinete Eletrico
Curitiba
Curitiba Paraná
Bicicleta
Patinete Eletrico
Florianópolis
Florianópolis Santa Catarina
Bicicleta
Patinete Eletrico
Fortaleza
Fortaleza Ceará
Bicicleta
Bicicleta
Goiânia
Goiânia Goiás
Bicicleta
Ilhabela
Ilhabela São Paulo
Bicicleta
Juiz de Fora
Juiz de Fora Minas Gerais
Bicicleta
Manaus
Manaus Amazonas
Bicicleta
Maricá
Maricá Rio de Janeiro
Bicicleta
Pelotas
Pelotas Rio Grande do Sul
Bicicleta
Petrolina
Petrolina Pernambuco
Bicicleta
Porto Alegre
Porto Alegre Rio Grande do Sul
cidade:  1
['Porto Alegre']
++++++++++++++++++
estado:  1
['Rio Grande do Sul']
++++++++++++++++++
região:  1
['Sul']
++++++++++++++++++
sistema:  1
['Bike PoA']
++++++++++++++++++
operadora:  1
['Tembici']
++++++++++++++++++
tipologia_do_veícul

sistema:  1
['Bike PE']
++++++++++++++++++
operadora:  1
['Tembici']
++++++++++++++++++
tipologia_do_veículo:  1
['Bicicleta']
++++++++++++++++++
estações:  1
[80.]
++++++++++++++++++
bicicletas_elétricas:  1
[nan]
++++++++++++++++++
bicicletas:  1
[800.]
++++++++++++++++++
patinetes_elétricos:  1
[nan]
++++++++++++++++++
viagens_diárias:  1
[2821.]
++++++++++++++++++
média_distância_percorrida_por_dia:  1
[14044.01]
++++++++++++++++++
usuários:  1
[nan]
++++++++++++++++++
mulheres:  1
[26.38]
++++++++++++++++++
homens:  1
[65.87]
++++++++++++++++++
15_até_29_anos:  1
[58.66]
++++++++++++++++++
30_até_59_anos:  1
[36.31]
++++++++++++++++++
acima_de_60:  1
[1.38]
++++++++++++++++++
emissões_de_co2_evitadas:  1
[74.34]
++++++++++++++++++
fonte_e_escopo_temporal:  1
['Emissões de CO² evitadas: Calculado com base na distância percorrida por dia. Metodologia do cálculo desenvolvida em colaboração com o Instituto de Energia e Meio Ambiente (IEMA). \nNº de estações, nº de veículos, viagens di

operadora:  1
['Tembici']
++++++++++++++++++
tipologia_do_veículo:  1
['Bicicleta']
++++++++++++++++++
estações:  1
[80.]
++++++++++++++++++
bicicletas_elétricas:  1
[nan]
++++++++++++++++++
bicicletas:  1
[800.]
++++++++++++++++++
patinetes_elétricos:  1
[nan]
++++++++++++++++++
viagens_diárias:  1
[2821.]
++++++++++++++++++
média_distância_percorrida_por_dia:  1
[14044.01]
++++++++++++++++++
usuários:  1
[nan]
++++++++++++++++++
mulheres:  1
[26.38]
++++++++++++++++++
homens:  1
[65.87]
++++++++++++++++++
15_até_29_anos:  1
[58.66]
++++++++++++++++++
30_até_59_anos:  1
[36.31]
++++++++++++++++++
acima_de_60:  1
[1.38]
++++++++++++++++++
emissões_de_co2_evitadas:  1
[74.34]
++++++++++++++++++
fonte_e_escopo_temporal:  1
['Emissões de CO² evitadas: Calculado com base na distância percorrida por dia. Metodologia do cálculo desenvolvida em colaboração com o Instituto de Energia e Meio Ambiente (IEMA). \nNº de estações, nº de veículos, viagens diárias, distância percorrida por dia, sexo e

cidade:  1
['Rio de Janeiro']
++++++++++++++++++
estado:  1
['Rio de Janeiro']
++++++++++++++++++
região:  1
['Sudeste']
++++++++++++++++++
sistema:  1
['Bike Rio']
++++++++++++++++++
operadora:  1
['Tembici']
++++++++++++++++++
tipologia_do_veículo:  1
['Bicicleta']
++++++++++++++++++
estações:  1
[260.]
++++++++++++++++++
bicicletas_elétricas:  1
[nan]
++++++++++++++++++
bicicletas:  1
[2600.]
++++++++++++++++++
patinetes_elétricos:  1
[nan]
++++++++++++++++++
viagens_diárias:  1
[17503.]
++++++++++++++++++
média_distância_percorrida_por_dia:  1
[97807.67]
++++++++++++++++++
usuários:  1
[nan]
++++++++++++++++++
mulheres:  1
[22.59]
++++++++++++++++++
homens:  1
[70.31]
++++++++++++++++++
15_até_29_anos:  1
[38.05]
++++++++++++++++++
30_até_59_anos:  1
[52.9]
++++++++++++++++++
acima_de_60:  1
[3.02]
++++++++++++++++++
emissões_de_co2_evitadas:  1
[63.53]
++++++++++++++++++
fonte_e_escopo_temporal:  1
['Emissões de CO² evitadas: Calculado com base na distância percorrida por dia. Met

['Bahia']
++++++++++++++++++
região:  1
['Nordeste']
++++++++++++++++++
sistema:  1
['Bike Salvador']
++++++++++++++++++
operadora:  1
['Tembici']
++++++++++++++++++
tipologia_do_veículo:  1
['Bicicleta']
++++++++++++++++++
estações:  1
[50.]
++++++++++++++++++
bicicletas_elétricas:  1
[nan]
++++++++++++++++++
bicicletas:  1
[400.]
++++++++++++++++++
patinetes_elétricos:  1
[nan]
++++++++++++++++++
viagens_diárias:  1
[1284.]
++++++++++++++++++
média_distância_percorrida_por_dia:  1
[8263.34]
++++++++++++++++++
usuários:  1
[nan]
++++++++++++++++++
mulheres:  1
[28.55]
++++++++++++++++++
homens:  1
[65.83]
++++++++++++++++++
15_até_29_anos:  1
[40.61]
++++++++++++++++++
30_até_59_anos:  1
[51.13]
++++++++++++++++++
acima_de_60:  1
[3.95]
++++++++++++++++++
emissões_de_co2_evitadas:  1
[38.3]
++++++++++++++++++
fonte_e_escopo_temporal:  1
['Emissões de CO² evitadas: Calculado com base na distância percorrida por dia. Metodologia do cálculo desenvolvida em colaboração com o Instituto de 

cidade:  1
['Vila Velha']
++++++++++++++++++
estado:  1
['Espírito Santo']
++++++++++++++++++
região:  1
['Sudeste']
++++++++++++++++++
sistema:  1
['Bike VV']
++++++++++++++++++
operadora:  1
['Tembici']
++++++++++++++++++
tipologia_do_veículo:  1
['Bicicleta']
++++++++++++++++++
estações:  1
[20.]
++++++++++++++++++
bicicletas_elétricas:  1
[nan]
++++++++++++++++++
bicicletas:  1
[200.]
++++++++++++++++++
patinetes_elétricos:  1
[nan]
++++++++++++++++++
viagens_diárias:  1
[2059.]
++++++++++++++++++
média_distância_percorrida_por_dia:  1
[10686.35]
++++++++++++++++++
usuários:  1
[nan]
++++++++++++++++++
mulheres:  1
[30.91]
++++++++++++++++++
homens:  1
[59.6]
++++++++++++++++++
15_até_29_anos:  1
[37.46]
++++++++++++++++++
30_até_59_anos:  1
[47.25]
++++++++++++++++++
acima_de_60:  1
[3.24]
++++++++++++++++++
emissões_de_co2_evitadas:  1
[71.3]
++++++++++++++++++
fonte_e_escopo_temporal:  1
['Emissões de CO² evitadas: Calculado com base na distância percorrida por dia. Metodologia 

In [37]:
dicionario_final

{'local': 'Brasil',
 'sistema_total': ['Bike Belém – PA',
  'Bike BH – MG',
  'Yellow – MG',
  'Unimed Bike JF – MG',
  'Rivibike – SP',
  'Yellow – SP',
  'Bike Santos – SP',
  'Bike Kids Santos – SP',
  'Bike Sanja – SP',
  'Mobilicidade – SP',
  'Grin – SP',
  'Lime – SP',
  'CicloSampa – SP',
  'Bike Sampa – SP',
  'Integrabike – SP',
  'Mais Bike – DF',
  'Yellow – DF',
  'Yellow – PR',
  'Yellow – SC',
  'Bicicleta Integrada – CE',
  'Bicicletar – CE',
  'GynDebike – GO',
  'Manôbike – AM',
  'Maricá – RJ',
  'Grin – RJ',
  'Lime – RJ',
  'Bike Rio – RJ',
  'Yellow – RJ',
  'Bike Pel – RS',
  'Bike PoA – RS',
  'Yellow – RS',
  'Petrobike – PE',
  'Bike PE – PE',
  'Salvador Vai de Bike – BA',
  'Bike Salvador – BA',
  'Bike Serra – ES',
  'Bike VV – ES',
  'Yellow – ES',
  'Bike Vitória – ES'],
 'sistema_por_estado': [{'estado': 'Pará', 'sistemas': ['Bike Belém – PA']},
  {'estado': 'Minas Gerais',
   'sistemas': ['Bike BH – MG', 'Yellow – MG', 'Unimed Bike JF – MG']},
  {'estad

In [48]:
with open('final_file_labmob_v7.json', 'w') as json_file:
  json.dump(dicionario_final, json_file, encoding='utf-16',ignore_nan=True,ensure_ascii =False)